# Tutorial 5: PO efficiencies and metrics.
In this tutorial, we will simulate a simple Cassegrain reflector telescope. We demonstrate how PyPO can be used to calculate some common figures of merit, such as efficiencies and far-field parameters.

In [ ]:
%matplotlib notebook

import numpy as np

from src.PyPO.System import System

s = System()

In [ ]:
parabola = {
            "name"      : "primary",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "vertex"    : np.zeros(3),
            "focus_1"   : np.array([0, 0, 3.5e3]),
            "lims_u"    : np.array([300, 5e3]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([1801, 801])
            }

s.addParabola(parabola)

hyperbola = {
            "name"      : "secondary",
            "pmode"     : "focus",
            "gmode"     : "uv",
            "flip"      : True,
            "focus_1"   : np.array([0, 0, 3.5e3]),
            "focus_2"   : np.array([0, 0, -1e3]),
            "ecc"       : 1.1,
            "lims_u"    : np.array([0, 300]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([1501, 501])
            }

s.addHyperbola(hyperbola)

focus = {
            "name"      : "focus",
            "gmode"     : "uv",
            "lims_u"    : np.array([0, 10]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([101, 101])
            }

s.addPlane(focus)

farfield = {
            "name"      : "farfield",
            "gmode"     : "AoE",
            "lims_u"    : np.array([-0.02, 0.02]),
            "lims_v"    : np.array([-0.02, 0.02]),
            "gridsize"  : np.array([101, 101])
            }

s.addPlane(farfield)

GPODict = {                                                                                                                                                                     
            "name"      : "source",                                                                                                                                  
            "lam"       : 1,                                                                                                      
            "w0x"       : 5,                                                                                             
            "w0y"       : 5,                                                                                             
            "n"         : 1,                                                                                                                             
            "E0"        : 1,                                                                                                                                  
            "dxyz"      : 0,                                                                                                 
            "pol"       : np.array([1, 0, 0])                                                                                                          
}

s.createGaussian(GPODict, "focus")
s.translateGrids("focus", np.array([0, 0, -1e3]))

We start by defining the Cassegrain reflectors. we also place a Gaussian beam in the lower focus of the Cassegrain. We define the Gaussian such that the secondary has an edge taper of roughly -11 dB. 

Now we are going to calculate all currents on the reflectors and the far-field. WARNING: the following code might take alot of time, especially if you are not running CUDA.

In [ ]:
source_to_sec = {
        "t_name"    : "secondary",
        "s_current" : "source",
        "epsilon"   : 10,
        "mode"      : "JM",
        "name_JM"   : "JM_sec",
        }

s.runPO(source_to_sec)

sec_to_pri = {
        "t_name"    : "primary",
        "s_current" : "JM_sec",
        "epsilon"   : 10,
        "mode"      : "JM",
        "name_JM"   : "JM_pri",
        }

s.runPO(sec_to_pri)

pri_to_ff = {
        "t_name"    : "farfield",
        "s_current" : "JM_pri",
        "epsilon"   : 10,
        "mode"      : "FF",
        "name_JM"   : "EH_ff",
        }

s.runPO(pri_to_ff)

In [ ]:
s.plotBeam2D("EH_ff", "Ex", vmin=-30)

Now that we have obtained currents on all reflectors and the far-field, we can proceed with the efficiencies calculation. We start by calculating the spillover efficiency on the secondary mirror. For that, we define a plane at the rim height of the secondary. We find the rim height by generating the reflectorgrids and using numpy to find the maximum z co-ordinate. The plane we define is oversized with respect to the secondary radius. This is done in order to capture as much illumination as possible on the plane.

In [ ]:
grids_sec = s.generateGrids("secondary")

h_rim = np.max(grids_sec.z)

rim_hyperbola = {
            "name"      : "rim_sec",
            "gmode"     : "uv",
            "flip"      : True,
            "lims_u"    : np.array([0, 300]) * 2,
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([1501, 501])
            }

s.addPlane(rim_hyperbola)
s.translateGrids("rim_sec", np.array([0, 0, h_rim]))

source_to_rim_sec = {
        "t_name"    : "rim_sec",
        "s_current" : "source",
        "epsilon"   : 10,
        "mode"      : "EH",
        "name_EH"   : "EH_rim_sec"
        }

s.runPO(source_to_rim_sec)

aper_sec = {
        "plot"      : True,
        "center"    : np.zeros(2),
        "outer"     : np.array([300, 300]),
        "inner"     : np.zeros(2)
}

s.plotBeam2D("EH_rim_sec", "Ex", vmin=-30, aperDict=aper_sec)

The spillover efficiency is calculated by defining an aperture (called 'aper_sec' here) on which to calculate spillover losses. The aperture is again a dictionary containing several fields. For an explanation, please have a look in the Templates.py file.

In [ ]:
eta_s = s.calcSpillover("EH_rim_sec", "Ex", aper_sec)
print(f"Spillover efficiency on secondary: {eta_s:.3f}")

The taper efficiency is calculated in the primary aperture. For this we define a plane in the focus of the primary, with the same radius as the primary mirror itself.

In [ ]:
foc_pri = {
            "name"      : "foc_pri",
            "gmode"     : "uv",
            "lims_u"    : np.array([300, 5e3]),
            "lims_v"    : np.array([0, 360]),
            "gridsize"  : np.array([1501, 501])
            }

s.addPlane(foc_pri)
s.translateGrids("foc_pri", np.array([0, 0, 3.5e3]))

pri_to_foc_pri = {
        "t_name"    : "foc_pri",
        "s_current" : "JM_pri",
        "epsilon"   : 10,
        "mode"      : "EH",
        "name_EH"   : "EH_foc_pri"
        }

s.runPO(pri_to_foc_pri)

s.plotBeam2D("EH_foc_pri", "Ex", vmin=-30)

In [ ]:
eta_t = s.calcTaper("EH_foc_pri", "Ex")
print(f"Taper efficiency in primary aperture: {eta_t:.3f}")